In [92]:
dir='/home/lisha/school/Projects/hyperband_nnet/hyperband2/mnist_svm/'
hyperband='hyperband/'
random='random/'
smac='smac/'
hyperopt='hyperopt/'
spearmint='spearmint/'
smac_early='smac_early/'
random_early='random_early'
hyperband_constant='hyperband_constant/'
fabolas='full_fabolas/'

In [93]:
import pickle
import glob, os
import fnmatch
import numpy

In [133]:
searchers={'random':random,'hyperband_constant':hyperband_constant,'fabolas':fabolas}
filenames={'hyperband':'hyperband','random':'random_hyperopt_august2013_mod', 'smac':'smac_2_06_01-dev',
           'hyperopt': 'hyperopt_august2013_mod','spearmint':'spearmint_april2013_mod','hyperband_constant':'hyperband',
           'random_early':'random_hyperopt_august2013_mod', 'smac_early':'smac_2_06_01-dev','fabolas':'results'}
results={}
for s in searchers.keys():
    subdir=dir+searchers[s]
    os.chdir(subdir)
    results[s]={}
    for dirpath, dirnames, files in os.walk(subdir):
        if s.find('early') < 0:
            if s =='hyperband' or s=='hyperband_constant':
                for f in fnmatch.filter(files, 'hyperband_run.log'):
                    index_trial=dirpath.index('trial')+5
                    seed=int(dirpath[index_trial:])
                    results[s][seed]=dirpath+'/'+f
            elif s=='fabolas':
                for f in fnmatch.filter(files, 'results.csv'):
                    index_trial=dirpath.index('seed')+4
                    seed=int(dirpath[index_trial:])
                    results[s][seed]=[dirpath+'/'+f]
                for f in fnmatch.filter(files, 'full_evals.pkl'):
                    index_trial=dirpath.index('seed')+4
                    seed=int(dirpath[index_trial:])
                    results[s][seed].append(dirpath+'/'+f)
            elif s=='random':
                for f in fnmatch.filter(files, 'results.pkl'):
                    index_trial=dirpath.index('trial')+5
                    seed=int(dirpath[index_trial:])
                    results[s][seed]=dirpath+'/'+f
            else:
                for f in fnmatch.filter(files, s+'*.pkl'):            
                    index_filename= dirpath.index(filenames[s])+len(filenames[s])
                    seed=int(dirpath[index_filename+1:dirpath.find('_',index_filename+1)])            
                    results[s][seed]=dirpath+'/'+f
        else:
            for f in fnmatch.filter(files, 'run_log.txt'):            
                index_filename= dirpath.index(filenames[s])+len(filenames[s])
                seed=int(dirpath[index_filename+1:dirpath.find('_',index_filename+1)])            
                results[s][seed]=[dirpath+'/'+f]


In [134]:
results['fabolas'].pop(24000)

['/home/lisha/school/Projects/hyperband_nnet/hyperband2/mnist_svm/full_fabolas/seed24000/results.csv']

In [34]:
def get_hyperband_results(dictionary,s):
    data=dictionary[0]
    hyperband_results=[]
    val_err=1
    test_err=1
    #times=[4, 8, 12, 16, 20,28,36,44,52,60,68]
    if s=='hyperband':
        times=[4, 8, 12, 16,24,32,40,48,56]
        indices=[0,1,2,3,5,6,7,8,9]
    else:
        times=[5,10,15,20,25,30]
        indices=[0,1,2,3,4,5]
    for i in range(len(times)):
        time=times[i]
        index=indices[i]
        if 1- data[index][1][-2] < val_err:
            val_err = 1- data[index][1][-2]
            test_err = 1-data[index][1][-1]
        hyperband_results.append([time,val_err,test_err])
    return hyperband_results

In [154]:
def get_hyperband_text(filename,full=True):
    results=[]
    results.append([0,1,1])
    with open(filename,'rb') as fp:
        for line in fp:
            if full:
                chars=line.split(' ')
                if chars[0][0:2]=='k=':
                    val_acc=float(chars[2][8:-1])
                    test_acc=float(chars[3][9:])
                if chars[0]=='time':
                    time=float(chars[2])
                    results.append([time,1-val_acc,1-test_acc])
                    
            else:
                chars=line.split(' ')
                if len(chars)>2 and len(chars) < 7:
                    try:
                        int(chars[0])
                        chars=[float(s) for s in chars]
                        results.append([results[-1][0]+chars[4],1-chars[2],1-chars[3]])
                    except ValueError:
                        continue
    return results

In [9]:
def get_random_results(data,speed=1):
    #data=dictionary[0]
    random_results=[]
    n_arms=len(data)
    val_err=1
    test_err=1
    
    for i in range(n_arms):
        time = (i+1)*1.0/speed
        if data[i]['result']<val_err:
            val_err = data[i]['result']
            test_err = data[i]['test_error']
        random_results.append([time,val_err,test_err])
    return random_results

In [10]:
def get_random(filename):
    data=pickle.load(open(filename,'rb'))
    results=[]
    results.append([0,1,1])
    for i in data:
        results.append([results[-1][0]+i[3],1-i[1],1-i[2]])
    return results

In [169]:
import csv
import ast
def get_fabolas(filename):
    data=[]
    data.append([0,1,1])
    with open(filename, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        i=0
        for row in spamreader:
            if i==0:
                i+=1
                continue                
            else:
                entry = ast.literal_eval(row[2])
                val_error=numpy.exp(entry[0])
                entry = ast.literal_eval(row[4])
                pred_error=numpy.exp(entry[0])
                time=float(row[7])/60.0
                data.append([time,val_error,pred_error])
    return data
def get_full_fabolas(filenames):
    csv_file=filenames[0]
    pickle_file=open(filenames[1],'rb')
    full_evals=pickle.load(pickle_file)
    for k in full_evals.keys():
        new_key=tuple([numpy.round(i,8) for i in k])
        full_evals[new_key]=full_evals[k]
        full_evals.pop(k)
    data=[]
    data.append([0,1,1])
    with open(csv_file, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        i=0
        for row in spamreader:
            if i==0:
                i+=1
                continue                
            else:
                entry = ast.literal_eval(row[2])
                val_error=numpy.exp(entry[0])
                entry = ast.literal_eval(row[4])
                pred_error=numpy.exp(entry[0])
                time=float(row[7])/60.0
                incumbent_str='['+','.join(row[3].split()[1:-1])+']'
                incumbent=ast.literal_eval(incumbent_str)
                incumbent=tuple(incumbent)
                if int(row[0])==39:
                    val_error=numpy.exp(full_evals[incumbent][0])
                    time=full_evals[incumbent][2]                    
                    data.append([time,val_error,pred_error])
                elif int(row[0])>39:
                    val_error=numpy.exp(full_evals[incumbent][0])                                
                    data.append([time,val_error,pred_error])
    csvfile.close()
    pickle_file.close()
    return data

In [75]:
test='[[ 0.78320247  0.20887057  1.        ]]'
'['+','.join(test.split()[1:-1])+']'

'[0.78320247,0.20887057,1.]'

In [143]:
def calculate_min(trial):
    intervals=range(5,24*60,5)
    results=[[0,1,1]]
    val_error=1
    test_error=1
    for t in trial:
        if t[1]<val_error:
            val_error=t[1]
            test_error=t[2]
        else:
            t[1]=val_error
            t[2]=test_error
    indices=range(len(trial))
    for i in intervals:
        index=[x for x in indices if trial[x][0]<i][-1]
        results.append([i,trial[index][1],trial[index][2]])
    return results
        
    

In [170]:
data_results={}
for s in results.keys():
    data_results[s]={}
    if s=='hyperband_constant':
        for seed in results[s].keys():
            data_results[s][seed]=calculate_min(get_hyperband_text(results[s][seed],True))
    elif s=='fabolas':
        for seed in results[s].keys():
            data_results[s][seed]=calculate_min(get_full_fabolas(results[s][seed]))
    else:
        for seed in results[s].keys():
            data_results[s][seed]=calculate_min(get_random(results[s][seed]))

In [163]:
data_results['fabolas']

{21000: [[0, 1, 1],
  [13.496445083618164, 0.031900000000000039, array([ 0.1283])],
  [19.317764202753704, 0.031900000000000039, array([ 0.14897939])],
  [23.581725084781645, 0.031900000000000039, array([ 0.08950131])],
  [39.9339183529218, 0.015700000000000051, array([ 0.06124943])],
  [43.937940235932665, 0.015700000000000051, array([ 0.05779383])],
  [58.007355415821074, 0.02380000000000004, array([ 0.05330308])],
  [62.26325303316116, 0.02380000000000004, array([ 0.04698979])],
  [79.62625548442205, 0.040699999999999965, array([ 0.03653458])],
  [83.58573381900787, 0.040699999999999965, array([ 0.03592052])],
  [88.48306571642557, 0.02380000000000004, array([ 0.0470142])],
  [93.36156163613002, 0.02380000000000004, array([ 0.04416097])],
  [98.03835210005442, 0.02380000000000004, array([ 0.0444828])],
  [114.80497648318608, 0.018700000000000046, array([ 0.04649147])],
  [120.4090713183085, 0.015700000000000051, array([ 0.04541958])],
  [125.05509131749471, 0.015700000000000051, arr

In [177]:
def calculate_stat(data):
    seeds=data.keys()
    indices=range(len(data[seeds[0]]))
    avg=[]
    for i in indices:
        time=data[seeds[0]][i][0]
        val_error = numpy.average([data[s][i][1] for s in seeds])
        test_error = numpy.average([data[s][i][2] for s in seeds])
        val_error_min = numpy.min([data[s][i][1] for s in seeds])
        test_error_min = numpy.min([data[s][i][2] for s in seeds])
        val_error_max = numpy.max([data[s][i][1] for s in seeds])
        test_error_max = numpy.max([data[s][i][2] for s in seeds])
        avg.append([time,[val_error_min,val_error,val_error_max],[test_error_min,test_error,test_error_max]])
    return avg

In [178]:
average_results={}
for s in data_results.keys():
    average_results[s] = calculate_stat(data_results[s])

In [109]:
average_results.keys()

['random', 'hyperband_constant', 'fabolas']

In [83]:
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt


In [190]:
colors={'hyperband_constant':'y','smac':'r','hyperopt':'b','random':'g','spearmint':'c','random_2x':'g','fabolas':'b'}
def results_plot(data):
    for s in data.keys():
        times=[i[0] for i in data[s]]
        mins=[i[1][0] for i in data[s]]
        avgs=[i[1][1] for i in data[s]]
        maxs=[i[1][2] for i in data[s]]
        if True:
            plt.plot(times,avgs,color=colors[s],label=s)            
            plt.plot(times,mins,color=colors[s],linestyle='--')
            plt.plot(times,maxs,color=colors[s],linestyle='--')
        if False:
            plt.semilogy(times,avgs,color=colors[s],label=s)            
            plt.semilogy(times,mins,color=colors[s],linestyle='--')
            plt.semilogy(times,maxs,color=colors[s],linestyle='--')
        #plt.fill_between(times,mins,maxs,alpha=0.4,color=colors[s])
    plt.legend()
    plt.xlabel('Minutes')
    plt.ylabel('Validation Error')
    plt.title("Average Validation Error")
pdf=PdfPages('../mnist_svm.pdf')
plt.figure(figsize=(6,6))
results_plot(average_results)
pdf.savefig()
plt.close()
pdf.close()


In [40]:
colors={'hyperband_constant':'y','smac':'r','hyperopt':'b','random':'g','spearmint':'c','random_2x':'g','smac_early':'r','random_early':'g'}
labels={'hyperband_constant':'hyperband','smac':'smac','hyperopt':'TPE','random':'random','spearmint':'spearmint','random_2x':'random_2x','smac_early':'smac_early','random_early':'random_early'}
def results_plot(index,error):
    matplotlib.rcParams.update({'font.size': 16})
    line_width=2
    edge_width=2
    shift = 0.3
    for s in ['random_early','smac_early','hyperband_constant']:
        if index==2:
            axis_label='Average Test Error'
        else:
            axis_label='Average Val Error'
        if s == 'hyperband' or s == 'hyperband_constant':
            if error:
                plt.errorbar([i[0][0] for i in mean_data[s]],[i[1][index] for i in mean_data[s]],color=colors[s],
                              yerr=[[i[0][index] for i in mean_data[s]],[i[2][index] for i in mean_data[s]]],
                         label=labels[s],linewidth=line_width,marker='x',markeredgewidth=edge_width,elinewidth=0.5,capthick=0.5)
            else:
                plt.plot([i[0][0] for i in mean_data[s]],[i[1][index] for i in mean_data[s]],color=colors[s],                          
                     label=labels[s],linewidth=line_width,marker='x',markeredgewidth=edge_width)
            #+' # trials = '+str(len(min_results[s].keys()))
            #x=[i[0][0] for i in mean_data[s]]
            #y=numpy.array([i[1][index] for i in mean_data[s]])
            #y_min=y-numpy.array([i[0][index] for i in mean_data[s]])
            #y_max=y+numpy.array([i[2][index] for i in mean_data[s]])
            #    plt.plot(x,y,color=colors[s],
            #              label=s+' # trials = '+str(len(min_results[s].keys())))
            #plt.fill_between(x,y_min,y_max,alpha=0.5,edgecolor='y',facecolor='y')
        elif s=='random_2x':
            if error:
                plt.errorbar([row[0][0] for row in mean_data[s]],[row[1][index] for row in mean_data[s]],color=colors[s],errorevery=8,
                          yerr=[[row[0][index] for row in mean_data[s]],
                                [row[2][index] for row in mean_data[s]]],label=labels[s],linewidth=line_width,marker='+',markeredgewidth=edge_width,elinewidth=0.5,capthick=0.5,markevery=4)
            else:
                plt.plot([row[0][0] for row in mean_data[s]],[row[1][index] for row in mean_data[s]],color=colors[s],
                          label=labels[s],linewidth=line_width,marker='+',markeredgewidth=edge_width,markevery=4)
        elif s=='smac_early' or s =='random_early':
            if error:
                plt.errorbar(numpy.array([row[0][0]+shift for row in mean_data[s]]),[row[1][index] for row in mean_data[s]],color=colors[s],errorevery=4,
                          yerr=[[row[0][index] for row in mean_data[s]],
                                [row[2][index] for row in mean_data[s]]],label=labels[s],linewidth=line_width,elinewidth=0.5,capthick=0.5,marker='^',markevery=4,markeredgecolor=colors[s])
            else:
                plt.plot(numpy.array([row[0][0] for row in mean_data[s]]),[row[1][index] for row in mean_data[s]],color=colors[s],
                          label=labels[s],linewidth=line_width, marker='^',markevery=4,markeredgecolor=colors[s])
            shift+=0.3
        else:
            #plt.plot([i[0][0] for i in mean_data[s]],[i[1][index] for i in mean_data[s]],color=colors[s],                          
            #         label=s+' # trials = '+str(len(min_results[s].keys())))
            #x=[row[0][0] for row in mean_data[s]]
            #y=numpy.array([row[1][index] for row in mean_data[s]])
            #y_min=y-numpy.array([row[0][index] for row in mean_data[s]])
            #y_max=y+numpy.array([row[2][index] for row in mean_data[s]])
            if error:
                plt.errorbar(numpy.array([row[0][0]+shift for row in mean_data[s]]),[row[1][index] for row in mean_data[s]],color=colors[s],errorevery=4,
                          yerr=[[row[0][index] for row in mean_data[s]],
                                [row[2][index] for row in mean_data[s]]],label=labels[s],linewidth=line_width,elinewidth=0.5,capthick=0.5)
            else:
                plt.plot(numpy.array([row[0][0] for row in mean_data[s]]),[row[1][index] for row in mean_data[s]],color=colors[s],
                          label=labels[s],linewidth=line_width)
            shift+=0.3
    plt.legend(fancybox=True, framealpha=0.75,loc=1)
    plt.xlim([0,31])
    #plt.ylim([0.025,0.175])
    plt.xlabel('Multiple of Max Iter Used')
    plt.ylabel(axis_label)
    #plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
pdf=PdfPages('../cifar100_small_search.pdf')
plt.figure(figsize=(6.5,5.5))
#plt.subplot(121)
#results_plot(1)
#plt.subplot(122)
results_plot(2,1)
#plt.suptitle('Test Error on CIFAR-10')
pdf.savefig()
plt.close()
pdf.close()


In [299]:
shift

NameError: name 'shift' is not defined

In [171]:
[(max([i[2] for i in data['random'][s][0]]),max([i[3] for i in data['random'][s][0]])) for s in data['random'].keys()]

[(array(0.7599999904632568, dtype=float32),
  array(0.8199999928474426, dtype=float32)),
 (array(0.7099999785423279, dtype=float32), array(0.75, dtype=float32)),
 (array(0.7099999785423279, dtype=float32),
  array(0.8100000023841858, dtype=float32)),
 (array(0.699999988079071, dtype=float32),
  array(0.7799999713897705, dtype=float32)),
 (array(0.699999988079071, dtype=float32),
  array(0.7099999785423279, dtype=float32)),
 (array(0.7799999713897705, dtype=float32),
  array(0.8299999833106995, dtype=float32)),
 (array(0.6800000071525574, dtype=float32),
  array(0.7900000214576721, dtype=float32)),
 (array(0.699999988079071, dtype=float32),
  array(0.7900000214576721, dtype=float32)),
 (array(0.7900000214576721, dtype=float32),
  array(0.8299999833106995, dtype=float32))]